In [1]:
from sklearn.datasets import load_iris
import pandas as pd


# Load the iris dataset
iris = load_iris()

# Create a DataFrame
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Display the first few rows
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [2]:
# is my data balanced or imbalanced?
df.target.value_counts()

target
0    50
1    50
2    50
Name: count, dtype: int64

In [3]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Total missing values in the dataset
total_missing = df.isnull().sum().sum()
print(f"\nTotal missing values: {total_missing}")

# Alternatively, get a summary of data types and non-null counts
df.info()

Missing values per column:
sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
target               0
dtype: int64

Total missing values: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [4]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [5]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = df.drop('target', axis=1)
y = df['target']

# First split: train and temp (temp will be split into validation and test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

# Second split: validation and test from temp
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Check sizes
print(f"Train set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


Train set: 90 samples
Validation set: 30 samples
Test set: 30 samples


In [7]:
import mlflow
import mlflow.sklearn
import shutil

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import os 

# Remove corrupted mlruns directory if it exists
if os.path.exists("./mlruns"):
    shutil.rmtree("./mlruns")

# Create mlruns directory
os.makedirs("./mlruns", exist_ok=True)
# Set up local tracking URI (creates a 'mlruns' folder in your current directory)
mlflow.set_tracking_uri("file:./mlruns")

# Create or set an experiment
mlflow.set_experiment("Iris Baseline Experiment")

solver = ["lbfgs", "newton-cg", "sag", "saga"]

for s in solver:
    # Start an MLflow run for each solver
    with mlflow.start_run():
        # Initialize and train the model
        model = LogisticRegression(random_state=42, max_iter=200, solver=s)
        model.fit(X_train, y_train)

        # Predict on validation set
        y_val_pred = model.predict(X_val)
        print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
        print("Validation Classification Report:")
        print(classification_report(y_val, y_val_pred))

        # Predict on test set
        y_test_pred = model.predict(X_test)
        print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
        print("Test Classification Report:")
        print(classification_report(y_test, y_test_pred))
        
        # Log parameters
        mlflow.log_param("model_type", "LogisticRegression")
        mlflow.log_param("solver", s)
        mlflow.log_param("random_state", 42)
        mlflow.log_param("max_iter", 200)
        
        # Log metrics
        mlflow.log_metric("test_accuracy", accuracy_score(y_test, y_test_pred))
        mlflow.log_metric("test_precision_macro", classification_report(y_test, y_test_pred, output_dict=True)['macro avg']['precision'])
        mlflow.log_metric("test_recall_macro", classification_report(y_test, y_test_pred, output_dict=True)['macro avg']['recall'])
        mlflow.log_metric("test_f1_macro", classification_report(y_test, y_test_pred, output_dict=True)['macro avg']['f1-score'])
        
        # Log the model
        mlflow.sklearn.log_model(model, "model")
        
        # Get the run ID for registration
        run_id = mlflow.active_run().info.run_id

    # Register the model outside the run context
    mlflow.register_model(f"runs:/{run_id}/model", f"iris_baseline_{s}")

/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/02/20 12:14:05 INFO mlflow.tracking.fluent: Experiment with name 'Iris Baseline Experiment' does not exist. Creating a new experiment.
2026/02/20 12:14:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Validation Accuracy: 0.9333333333333333
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

Test Accuracy: 0.9666666666666667
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.91      1.00      0.95        10
           2       1.00      0.90      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/mlflow/tracking/_model_registry/utils.py:216: FutureWarning: The filesystem model registry backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow

Validation Accuracy: 0.9333333333333333
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30

Test Accuracy: 0.9666666666666667
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.91      1.00      0.95        10
           2       1.00      0.90      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Successfully registered model 'iris_baseline_newton-cg'.
2026/02/20 12:14:15 WARNING mlflow.tracking._model_registry.fluent: Run with id 40a10f77ec044287b8671e522404b0eb has no artifacts at artifact path 'model', registering model based on models:/m-2b06f04f9ae04ea9ace6c16ed2bc3993 instead
Created version '1' of model 'iris_baseline_newton-cg'.
/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
2026/02/20 12:14:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arb

Validation Accuracy: 1.0
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Test Accuracy: 0.9666666666666667
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.91      1.00      0.95        10
           2       1.00      0.90      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Successfully registered model 'iris_baseline_sag'.
2026/02/20 12:14:17 WARNING mlflow.tracking._model_registry.fluent: Run with id 82cee95de91142a9a0e721a3d9f60a47 has no artifacts at artifact path 'model', registering model based on models:/m-18b918fc5dc04664b2fd84e9fdc982cc instead
Created version '1' of model 'iris_baseline_sag'.
/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
2026/02/20 12:14:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/iashahba/Documents/ARTIFIN/FS26/SW2/Data and Code/.venv/lib/python3.14/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code 

Validation Accuracy: 1.0
Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Test Accuracy: 1.0
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



Successfully registered model 'iris_baseline_saga'.
2026/02/20 12:14:20 WARNING mlflow.tracking._model_registry.fluent: Run with id 327e797730e3460d96ce6b1457a26a8c has no artifacts at artifact path 'model', registering model based on models:/m-67be64eb1701443890004d44e41691ab instead
Created version '1' of model 'iris_baseline_saga'.
